In [1]:
import sys
import argparse
import datetime
import re
import numpy as np
import pandas as pd
import random
import multiprocessing
import networkx as nx
import scipy as sp
from math import floor, ceil
from itertools import chain, product, repeat
from sklearn.cluster import KMeans
from tqdm.auto import tqdm, trange
import warnings
from multiprocessing import Pool
from collections import defaultdict
warnings.filterwarnings('ignore')

In [2]:
def get_overlaps(inds_1, inds_2): #verified.
    """
    Calculate the overlaps between two reads and the distance spanned by them.
    The reads are each assumed to comprise two arms, a left and a right arm.
    inds_1 and inds_2: [left start, left stop, right start, right stop]
    Returns overlap_l, overlap_r, span_l, span_r: int, int, int, int
    """
    overlap_l = min(inds_1[1], inds_2[1]) - max(inds_1[0], inds_2[0]) + 1
    overlap_r = min(inds_1[3], inds_2[3]) - max(inds_1[2], inds_2[2]) + 1
    span_l = max(inds_1[1], inds_2[1]) - min(inds_1[0], inds_2[0]) + 1
    span_r = max(inds_1[3], inds_2[3]) - min(inds_1[2], inds_2[2]) + 1
    return overlap_l, overlap_r, span_l, span_r

def get_overlap_ratios(inds_1, inds_2): #verified.
    """
    Function to calculate the overlap ratio between two reads
    inds_1 and inds_2: [left start, left stop, right start, right stop]
    Returns ratio_l, ratio_r : float, float
    The integer divisions are not backward compatible with python2. 
    """
    overlap_l, overlap_r, span_l, span_r = get_overlaps(inds_1, inds_2)
    ratio_l = overlap_l / span_l; ratio_r = overlap_r / span_r
    return ratio_l, ratio_r

covlimit = 100
def subsample(genealign, covlimit): #requires random. 
    #genealign: [[alignid,info1,info2]...]. info:chrom,strand,start,end,line
    cov={}; nalign=len(genealign)
    # get the list of [[chrom, srrabd, start, end] * n]
    intvls = [i[1:5] for i in genealign] + [i[6:10] for i in genealign] 
    # create a dict (chrom, strand, start/end):count
    for intvl in intvls:
        for i in intvl[2:4]:
            if (*intvl[:2],i) not in cov: cov[(*intvl[:2],i)]=1
            else: cov[(*intvl[:2],i)]+=1
    # the max of start/end repetitive counts
    maxcov=max(list(cov.values()))
    genealignlimit = genealign #first set the limited list to all alignments
    geneextra = [] #store nonselected alignments.
    # if max of start/end repetitive counts is greater than the user set limit
    # randomly select int(nalign*covlimit/maxcov) entries, exclude then and save in geneextra
    if maxcov>covlimit: 
        random.seed(0)
        idx=random.sample(range(nalign),int(nalign*covlimit/maxcov))
        geneextra = [genealign[i] for i in range(nalign) if i not in idx]
        genealignlimit = [genealign[i] for i in idx]
    return genealignlimit, geneextra

def graph_align(genealign, t): #requires networkx. only graph the sub sampled
    """
    Function that creates a weighted graph representation of the alignments
    Alignments are represented by nodes. Two alignments with left and right
    overlap ratios > threshold t are connected by an edge of weight overlap/span
    input genealign is one item from the genealigndict dictionary.
    (gene1,gene2):[[alignid,info1,info2]...]. info:chrom,strand,start,end,line
    alignids: np array. Read IDs
    genealign: list of alignments
    t: float. Overlap threshold, default 0.1 for cliques and 0.5 for spectral
    Returns graph: NetworkX graph
    requires networkx, get_overlap_ratios(), etc.
    """
    graph = nx.Graph()
    alignids = [i[0] for i in genealign]
    alignindsdict = dict([(i[0],i[3:5]+i[8:10]) for i in genealign])
    #alignindsdict format: (alignid: [start1, end1, start2, end2])
    graph.add_nodes_from(alignids)
    # just the [start1, end1, start2, end2] from alignindsdict
    aligninds = np.array(list(alignindsdict.values()))
    sorted_ids = []
    for i in range(4): sorted_ids.append([alignid for (ind, alignid) in
                                          sorted(zip(aligninds[:,i],alignids))])
    for (id_1, inds_1) in tqdm(zip(alignids, aligninds), total = len(alignids)):
        for i in range(4):
            id_list = sorted_ids[i]; j = id_list.index(id_1); loop_flag = 1
            if i%2 == 0: check_loop = (j < len(aligninds) - 1)
            else: check_loop = (j > 0)
            while (loop_flag == 1) and check_loop:
                if i%2 == 0: j += 1; check_loop = (j < len(aligninds) - 1)
                else: j -= 1; check_loop = (j > 0)
                id_2 = id_list[j]; inds_2 = alignindsdict[id_2]
                if i == 0: check_inds = (inds_2[0] <= inds_1[1])
                elif i == 1: check_inds = (inds_2[1] >= inds_1[0])
                elif i == 2: check_inds = (inds_2[2] <= inds_1[3])
                else: check_inds = (inds_2[3] >= inds_1[2])
                if check_inds:
                    ratio_l, ratio_r = get_overlap_ratios(inds_1, inds_2)
                    if (ratio_l > t) and (ratio_r > t): 
                         graph.add_edge(id_1, id_2, weight=(ratio_l + ratio_r))
                else: loop_flag = 0
    return graph

def get_spectral(graph, n=10, t=5): 
    """
    Function that performs spectral clustering on the weighted graph.
    Cluster number, k, is determined by finding the first eigengap that is
    some amount t larger than preceding eigengaps.
    graph : NetworkX graph. Weighted graph representation of all alignments
    n: int. Number of eigenvalues (DG splits) to consider threshold
    t: int. Multiplicity of median eigengap threshold
    Returns align_dg_dict, dg_ind: dict, int
    requires KMeans, networkx functions:
    connected_components(),subgraph.nodes(), subgraph.degree(), etc.  
    """
    dg_ind = 0; align_dg_dict = {}
    subgraphs = [graph.subgraph(c) for c in nx.connected_components(graph)]
    for subgraph in tqdm(subgraphs, total = len(subgraphs)):
        k = 1
        if len(subgraph) > 1:
            L=nx.laplacian_matrix(
                subgraph,nodelist=sorted(subgraph.nodes())).todense()
            D=np.diag([subgraph.degree[node]
                       for node in sorted(subgraph.nodes())])
            w, v = sp.linalg.eigh(L, D, type=1)  # Since L always symmetric
            eigengaps = np.diff(w[:(n + 1)])
            if len(eigengaps) > 2:
                if (w[1] > 1) and (w[1] >= 10*np.median(eigengaps[1:])): k = 2
                else:
                    # ignore divide by 0 warning if eigengaps median is 0
                    np.seterr(divide='ignore', invalid='ignore')
                    eigenratios = np.copy(eigengaps)
                    eigenratios[1:] = np.array([
                        eigengaps[i] / np.median(eigengaps[:i])
                        for i in range(1, len(eigengaps))])
                    if max(eigenratios) >= t: k = np.argmax(eigenratios>=t)+2
            Y = np.transpose(v[:k])
            kmeans = KMeans(n_clusters=k, random_state=0).fit(Y)
            kmeans.labels_ += dg_ind
            subgraph_dict = dict(zip(sorted(subgraph.nodes()), kmeans.labels_))
        else: subgraph_dict = {list(subgraph)[0]: dg_ind}
        align_dg_dict = {**align_dg_dict, **subgraph_dict}; dg_ind += k
    return align_dg_dict #asignment of alignment IDs to DG numbers.

def get_cliques(graph):
    """
    Function that gets cliques from subgraphs of connected components.
    graph : NetworkX graph. Weighted graph representation of all alignments
    Returns align_dg_dict : dict, int
    requires networkx (<2.4) functions:
    connected_components(), subgraph(), find_cliques, etc. 
    """
    dg_ind = 0; align_dg_dict = {}
    subgraphs = [graph.subgraph(c) for c in nx.connected_components(graph)]
    for subgraph in tqdm(subgraphs, total = len(subgraphs)):
        if len(subgraph) > 1:
            cliques_kept = []
            cliques_all = list(nx.find_cliques(subgraph))
            cliques_all.sort(key=len)
            while len(cliques_all) > 0:
                cliques_nodes=set(
                    [node for clique in cliques_kept for node in clique])
                clique_test = cliques_all.pop()
                if not set(list(clique_test)).intersection(cliques_nodes):
                    cliques_kept.append(clique_test)
            dg_inds=[[dg_ind+i]*len(clique)
                     for i,clique in enumerate(cliques_kept)]
            subgraph_dict = dict(
                zip([node for clique in cliques_kept for node in clique],
                    [ind for inds_list in dg_inds for ind in inds_list]))
            align_dg_dict = {**align_dg_dict, **subgraph_dict}
            dg_ind += len(cliques_kept)
        else: read_id=list(subgraph)[0];align_dg_dict[read_id]=dg_ind;dg_ind+=1
    return align_dg_dict #asignment of alignment IDs to DG numbers. 

def get_dgs(align_dg_dict):
    """
    Function that creates inverse dictionary of align_dg_dict
    align_dg_dict: dict. Dictionary of alignments and clustering DG assignments
    Returns dg_align_dict: dict, k=dg_id, v=[alignids]
    align_dg_dict comes from get_spectral(graph) or get_cliques(graph)
    """
    dgs_list = set(align_dg_dict.values()) #list of all duplex groups
    dg_align_dict = {}
    for dg in tqdm(dgs_list, total = len(dgs_list)):
        dg_align_list =[x for (x,y) in align_dg_dict.items() if y == dg]
        dg_align_dict[dg] = dg_align_list
    return dg_align_dict


def filter_dgs(dg_align_dict, genealign):
    """
    Function to filter out invalid DGs
    DGs whose alignments are all identical are eliminated.
    Here we compare (chr1,strand1,start1,end1,chr2,strand2,start2,end2)
    DGs with fewer than two alignments are also eliminated.
    genealign, format as follows
    (gene1,gene2):[[alignid,info1,info2]...]. info:chrom,strand,start,end,line
    dg_align_dict: Dictionary of DGs and their alignments
    Returns filtered_dict : dict
    """
    filtered_dict = {}
    for (dg, alignids) in tqdm(dg_align_dict.items(), total = len(dg_align_dict)):
        num_align = len(alignids)
        infolist = [tuple(i[1:5]+i[6:10]) for i in genealign]
        if num_align>1 and len(set(infolist))>1: filtered_dict[dg] = alignids
    return filtered_dict

def create_stats(dg_align_dict, genealign):
    """
    Function that creates the DG stats dictionary
    The DG stats dictionary no longer has individual alignments but instead
    contains metadata for each DG, including number of alignments, coverage, and
    nonoverlapping group (NG). Coverage fraction is defined as c/sqrt(a*b) where
        + c = number of alignments in a given DG
        + a = number of alignments overlapping the left arm of the DG
        + b = number of alignments overlapping the right arm of the DG
    dg_align_dict: dict. Dictionary of DGs and their alignments
    genealign: list of alignments to this (gene1,gene2) pair
    format: [[alignid,info1,info2]...]. info:chrom,strand,start,end,line
    Returns dg_stats_dict: dict
    """
    ng_ind = 0
    gene_align = list(chain.from_iterable(dg_align_dict.values()))#all alignids
    dg_stats_dict = {}; ng_dict = {}
    for (dg, alignids) in tqdm(dg_align_dict.items()):
        # Get DG indices: dg_min, dg_max and dg_inds (4 medians for each dg)
        dg1aligndict = dict.fromkeys(alignids)
        dg_min=min([i[3] for i in genealign if i[0] in dg1aligndict])
        dg_max=max([i[9] for i in genealign if i[0] in dg1aligndict])
        
        inds=np.array([i[3:5]+i[8:10]for i in genealign if i[0]in dg1aligndict])
        
        # change inds from median to min of the start and max of the right
        #dg_inds = [np.median(inds[:,0]), np.median(inds[:,1]), np.median(inds[:,2]), np.median(inds[:,3])]
        dg_inds = [np.min(inds[:,0]), np.max(inds[:,1]), np.min(inds[:,2]), np.max(inds[:,3])]
#         dg_inds = [int(i) for i in np.median(inds, axis=0)]

        #Calculate coverage based on non-continuous alignments gap1+rri
        chrom1,strand1,chrom2,strand2=tuple(genealign[0][1:3]+genealign[0][6:8])
        start1,end1,start2,end2=tuple(dg_inds)
#         range1=[i*10 for i in range(floor(int(start1)/10),ceil(int(end1)/10))]
#         range2=[i*10 for i in range(floor(int(start2)/10),ceil(int(end2)/10))]
#         overlap1=max([covdict[(chrom1,strand1,i)] for i in range1 if
#                       (chrom1,strand1,i) in covdict])
#         overlap2=max([covdict[(chrom2,strand2,i)] for i in range2 if
#                       (chrom2,strand2,i) in covdict])
#         covfrac=len(alignids)/np.sqrt(overlap1*overlap2)
        #print(dg,len(alignids),overlap1,overlap2,covfrac)

       
        # Assign NG for compact visualization in genome browsers e.g. IGV
        # ng_dict format: ng_id: [dg_id list]
        # ng_dg_align format: list of DG ids.
        # ng_dgs: list of dgs for each ng.
        # DGs in each (gene1,gene2) pair are processed separately.
        # To visualize alignments properly in IGV,
        # set "Sort alignments by tag DG" and "Group alignments by tag NG"
        if len(ng_dict)==0: ng_dict[ng_ind]=[dg]; dg_ng=ng_ind; ng_ind+=1
        else:
            ng_assigned = 0
            for (ng, ng_dgs) in ng_dict.items():
                ng_overlaps = np.zeros(len(ng_dgs))
                for (i, ng_dg) in enumerate(ng_dgs):
                    ng_dg_align = dict.fromkeys(dg_align_dict[ng_dg])
                    dginfo=[i for i in genealign if i[0] in ng_dg_align]
                    #dginfo: [[alignid,info1,info2]...].
                    #info:chrom,strand,start,end,line
                    ng_dg_min = min([i[3] for i in dginfo])
                    ng_dg_max = max([i[9] for i in dginfo])
                    overlap = min(dg_max, ng_dg_max) - max(dg_min, ng_dg_min)
                    if overlap > 0: ng_overlaps[i] = 1
                if sum(ng_overlaps) == 0:
                    ng_dict[ng].append(dg); dg_ng = ng; ng_assigned += 1; break
            if ng_assigned == 0:
                ng_dict[ng_ind] = [dg]; dg_ng = ng_ind; ng_ind += 1
        
        dg_stats_dict[dg]={'chromstrand': [chrom1,strand1,chrom2,strand2],
                           'dg_inds': dg_inds,
                           'num_align': len(alignids), 'NG': dg_ng}
    return dg_stats_dict

def writedg(outdg,dg_stats_dict,genepair): #bedpe format output
    """
    see bedpe definition here: 
    https://bedtools.readthedocs.io/en/latest/content/general-usage.html
    chrom1 start1 end1 chrom2 start2 end2 name score strand1 strand2 additional
    bedpe can be easily converted to the bed12 format to visualize arcs. 
    """
    with open(outdg, 'w+') as f:
        for (dg, dg_info) in dg_stats_dict.items():
            chromstrand,dg_inds = dg_info['chromstrand'],dg_info['dg_inds']
            num_align = dg_info['num_align']
            dg_str = '{0},{1},{2}'.format(genepair, dg, 0)
            line = [chromstrand[0], str(dg_inds[0]), str(dg_inds[1]),
                    chromstrand[2], str(dg_inds[2]), str(dg_inds[3]), dg_str, 
                    str(num_align), chromstrand[1], chromstrand[3]]
            f.write('\t'.join(line)+'\n')
    return

def get_graph(test_lst, t=0.2):
    '''
    the different version of the function graph_align
    format of test_lst
    [[alignid,info1,info2]...]. info:chrom,strand,start,end,line for two strands
    line can be anything
    alignids: np array. Read IDs
    '''
    alignids = [i[0] for i in test_lst]
    alignindsdict = dict([(i[0],i[3:5]+i[8:10]) for i in test_lst])

    graph = nx.Graph()
    alignindsdict_backup = {}
    alignids = list(alignindsdict.keys())
    # loop through the dictionary and discard the first kv pair in current dict after each iteration
    pbar = tqdm(total = len(alignids))
    while len(alignids)>0:
        ID = alignids[0]
    #     print(f'start with {ID}')
        v_0 = alignindsdict[ID]
        for k, v in alignindsdict.items():
            if ID != k:
                ratio_l, ratio_r = get_overlap_ratios(v_0, v)
                if (ratio_l > t) and (ratio_r > t): 
                    graph.add_edge(ID, k, weight=(ratio_l + ratio_r))
    #                 print(f'add edge {ID} and {k}')
    #                 print(f'ratios are {ratio_l} and {ratio_r}')
    #                 print(f'add edge {v_0} and {v}')
        alignids.pop(0)
        alignindsdict_backup[ID] = v
        del alignindsdict[ID]
        pbar.update(1)
    pbar.close()
    return graph

# Rewrite get_graph function

In [3]:
# Function for node creation with multiprocessing
def get_overlap_node_weight(i, filtered_df, t=0.8):
    '''
    function to generate connected nodes and weight in a graph
    given a df (filtered_df) that contains pair information (gene1, start1, end1, gene2, start2, end2)
    note the df should be reindexed, i is the index for each row
    t is the overlapping threshold, if two pairs overalap>t, they are conected in the graph
    out put is a list of index (ID, or nodes) that are considered overlapped -- overlap_ID
    and the weight associated with the overlapped nodes
    '''
    # start and end positions of this row
    start1, end1, start2, end2 = filtered_df.loc[i, ['start1', 'end1', 'start2', 'end2']]
    # slice the df to only contain the rows after index i
    df_to_compare = filtered_df[i+1:]
    # change here to all other rows
    # df_to_compare = filtered_df
    # # now calculate the overlap ratio between the row and df_to_compare
    mat_compare = df_to_compare[['start1', 'end1', 'start2', 'end2']].values
    # stats for the current row
    mat_current = np.tile([start1, end1, start2, end2], len(df_to_compare)).reshape(-1, 4)
    # calculate min, max values based on the get_overlap_ratios function
    min0, min1, min2, min3 = [np.minimum(mat_compare, mat_current)[:, i] for i in range(mat_current.shape[1])]
    max0, max1, max2, max3 = [np.maximum(mat_compare, mat_current)[:, i] for i in range(mat_current.shape[1])]
    ratio_l = (min1-max0+1) / (max1-min0+1)
    ratio_r = (min3-max2+1) / (max3-min2+1)
    # get index of entries that have overlapping ratio greater than t (0.7 here)
    list1 = [index for index,v in enumerate(ratio_l) if v > t]
    list2 = [index for index,v in enumerate(ratio_r) if v > t]
    # note here j+i+1 is to correct the index in list1 and list2
    # the starting index (0) in list1 and list2 is actually i+1 in the original filtered_df
    # because the two lists are from df_to_compare, which is filtered_df[i+1:]
    overlap_ID = [j+i+1 for j in list(set(list1).intersection(list2))]
    # change here to all other rows
    # overlap_ID = list(set(list1).intersection(list2))
    weight = [ratio_l[num-i-1] + ratio_r[num-i-1] for num in overlap_ID]
    # for num in overlap_ID:
    #     graph.add_edge(i, num, weight=(ratio_l[num] + ratio_r[num]))
    return overlap_ID, weight

In [4]:
def run_total_clustering(n, top_50_snoRNA, test_df, t, csv_name = 'dummy', 
                         ncore = 60, select = 'median', threshold = 20):
    '''
    function to generate a graph and run spectral clustering
    n is the index in top_50_snoRNA (snoRNA df, at least contain snoRNA name as the "name" column)
    test_df is the df contains pair information (gene1, start1, end1, gene2, start2, end2)
    t is the overlapping threshold, if two pairs overalap>t, they are conected in the graph
    csv_name is the output csv name
    ncore is the number of cores used for multiprocessing
    select is the way to pick data group a group, either median or minmax
    threshold is the minimum number of counts
    '''
    # 0. select snoRNA
    snoRNA = top_50_snoRNA.loc[n, 'name']
    # https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
    # 1. generate graph parameters (IDs that are overlapped and the weight)
    p = Pool(ncore)
    print(f'processing {snoRNA}...')
    filtered_df = test_df[(test_df['gene1'] == snoRNA)].reset_index()
    filtered_df = filtered_df.rename(columns={'level_0':'ID'})
    # generating graph parameters (connected nodes and weights)
    result = p.starmap(get_overlap_node_weight, zip(np.arange(len(filtered_df)), 
                                                    repeat(filtered_df), repeat(t)))

    # 2. create weighted graph
    print('creating weighted graph...')
    graph = nx.Graph()
    for i, v in tqdm(enumerate(result), total = len(result)):
        if v[0]:
            for j, k in enumerate(v[0]):
                graph.add_edge(i, k, weight=(v[1][j]))

    # 3. spectral clustering
    print('spectral clustering...')
    sprctal_cluster = get_spectral(graph)

    # 4. get dg dictionary 
    # in this dictionary, keys are the DG group numbers, values are the index in df that belong to this DG group
    print('get DG dictionary...')
    dg_align_dict = get_dgs(sprctal_cluster)

    # 5. get groups df, save csv
    # select rows in the filtered_df using indexes generated from clustering in the last step
    print('get DG output...')
    groups = []
    for k, v in dg_align_dict.items():
        group_df = filtered_df.loc[v]
        if select == 'median':
            start1, end1, start2, end2 = \
            np.median(group_df[['start1', 'end1', 'start2', 'end2']], axis = 0).astype(int)
        if select == 'minmax':
            start1, start2 = np.min(group_df[['start1', 'start2']], axis = 0).astype(int)
            end1, end2 = np.max(group_df[['end1', 'end2']], axis = 0).astype(int)
        if select == 'percentile':
            start1, start2 = \
            np.percentile(group_df[['start1', 'start2']], 25, axis = 0).astype(int)
            
            end1, end2 = \
            np.percentile(group_df[['end1', 'end2']], 75, axis = 0).astype(int)
        
        count = len(group_df)
        groups.append([start1, end1, start2, end2, count]) 
    output_df = pd.DataFrame(groups, columns = ['start1', 'end1', 'start2', 'end2', 'count'])
    output_df.insert(0, column='snoRNA', value=snoRNA)
    output_df.insert(3, column='target', value='45S_rRNA')
    output_df = output_df.sort_values(by = 'count', ascending = False)
    output_df = output_df[output_df['count']>=threshold]
    output_df.to_csv(csv_name + '.csv', index = False)
    
    # terminate Pool
    p.close()
    p.join()

In [5]:
def run_total_clustering_nonsnoRNA(filtered_df, t, target_name1, target_name2,
                                   csv_name = 'dummy', ncore = 60, 
                                   select = 'median', threshold = 20, save = True):
    '''
    function to generate a graph and run spectral clustering for non-snoRNA analysis
    filtered_df is the df contains pair information (gene1, start1, end1, gene2, start2, end2)
    t is the overlapping threshold, if two pairs overalap>t, they are conected in the graph
    csv_name is the output csv name
    ncore is the number of cores used for multiprocessing
    select is the way to pick data group a group, either median or minmax
    threshold is the minimum number of counts
    target_name1, target_name2 are the names of RNAs that interact
    '''
    # https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
    # 1. generate graph parameters (IDs that are overlapped and the weight)
    p = Pool(ncore)
    print(f'processing...')
    # generating graph parameters (connected nodes and weights)
    result = p.starmap(get_overlap_node_weight, zip(np.arange(len(filtered_df)), 
                                                    repeat(filtered_df), repeat(t)))

    # 2. create weighted graph
    print('creating weighted graph...')
    graph = nx.Graph()
    for i, v in tqdm(enumerate(result), total = len(result)):
        if v[0]:
            for j, k in enumerate(v[0]):
                graph.add_edge(i, k, weight=(v[1][j]))

    # 3. spectral clustering
    print('spectral clustering...')
    sprctal_cluster = get_spectral(graph)

    # 4. get dg dictionary 
    # in this dictionary, keys are the DG group numbers, values are the index in df that belong to this DG group
    print('get DG dictionary...')
    dg_align_dict = get_dgs(sprctal_cluster)
    
    # skip empty dict
    if dg_align_dict == {}:
        return None

    # 5. get groups df, save csv
    # select rows in the filtered_df using indexes generated from clustering in the last step
    print('get DG output...')
    dct = defaultdict(int)
    for k, v in dg_align_dict.items():
        group_df = filtered_df.loc[v]
        if select == 'median':
            start1, end1, start2, end2 = \
            np.median(group_df[['start1', 'end1', 'start2', 'end2']], axis = 0).astype(int)
        if select == 'minmax':
            start1, start2 = np.min(group_df[['start1', 'start2']], axis = 0).astype(int)
            end1, end2 = np.max(group_df[['end1', 'end2']], axis = 0).astype(int)
        if select == 'percentile':
            start1, start2 = \
            np.percentile(group_df[['start1', 'start2']], 25, axis = 0).astype(int)
            
            end1, end2 = \
            np.percentile(group_df[['end1', 'end2']], 75, axis = 0).astype(int)
        
        count = len(group_df)
        string = ('_').join([str(start1), str(end1), str(start2), str(end2)])
        dct[string] += count
    
    # make data frame
    dct_df = pd.DataFrame(dct.items())
    output_df = dct_df[0].str.split('_', expand = True)
    output_df[4]= dct_df[1]
    output_df.columns = ['start1', 'end1', 'start2', 'end2', 'count']
    output_df.insert(0, column='gene1', value=target_name1)
    output_df.insert(3, column='gene2', value=target_name2)
    output_df = output_df.sort_values(by = 'count', ascending = False)
    output_df = output_df[output_df['count']>=threshold]
    if save:
        output_df.to_csv(csv_name + '.csv', index = False)
    
    # terminate Pool
    p.close()
    p.join()
    return output_df